In [1]:
import json
from jsonschema import validate, ValidationError, SchemaError

def is_valid_json(json_string, json_schema):
    try:
        json_object = json.loads(json_string)
        validate(instance=json_object, schema=json_schema)
        return True
    except ValidationError as e:
        print(f"JSON validation failed: {e.message}")
        display_invalid_attributes(e, json_object)
        return False
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e.msg} at line {e.lineno}")
        print_error_location(json_string, e.lineno)
        return False
    except SchemaError as e:
        print(f"Schema error: {e.message}")
        return False

def display_invalid_attributes(error, instance):
    for error_path in error.path:
        if isinstance(error_path, int):
            attr = [a for a in ['items', 'ItemList', 'BchDtls', 'AttribDtls'] if error_path == instance.get(a)[error_path]]
            print(f"Invalid attribute: {attr[0]}")
        else:
            attr = error_path
            print(f"Invalid attribute: {attr}")

def print_error_location(json_string, error_line):
    lines = json_string.split('\n')
    print(f"Error occurred at line {error_line}:")
    print(lines[error_line - 1])  # Python lists are zero-indexed, hence -1

# Example usage:

json_schema = {
	"$schema": "http://json-schema.org/draft-07/schema#",
	"Title": "GST-India Invoice Document ",
	"Description": "GST Invoice format for IRN Generation in INDIA",
	"Version": {
		"type": "string",
		"minLength": 1,
		"maxLength": 6,
		"description": "Version of the schema"
	},
	"Irn": {
		"type": "string",
		"minLength": 64,
		"maxLength": 64,
		"description": "Invoice Reference Number"
	},
	"TranDtls": {
		"type": "object",
		"properties": {
			"TaxSch": {
				"type": "string",
				"minLength": 3,
				"maxLength": 10,
				"enum": [
					"GST"
				],
				"pattern": "^(GST)$",
				"description": "GST- Goods and Services Tax Scheme"
			},
			"SupTyp": {
				"type": "string",
				"minLength": 3,
				"maxLength": 10,
				"enum": [
					"B2B",
					"SEZWP",
					"SEZWOP",
					"EXPWP",
					"EXPWOP",
					"DEXP"
				],
				"pattern": "(?i)^((B2B)|(SEZWP)|(SEZWOP)|(EXPWP)|(EXPWOP)|(DEXP))$",
				"description": "Type of Supply: B2B-Business to Business, SEZWP - SEZ with payment, SEZWOP - SEZ without payment, EXPWP - Export with Payment, EXPWOP - Export without payment,DEXP - Deemed Export"
			},
			"RegRev": {
				"type": "string",
				"minLength": 1,
				"maxLength": 1,
				"enum": [
					"Y",
					"N"
				],
				"pattern": "^([Y|N]{1})$",
				"description": "Y- whether the tax liability is payable under reverse charge"
			},
			"EcmGstin": {
				"type": "string",
				"minLength": 15,
				"maxLength": 15,
				"pattern": "^([0-9]{2}[0-9A-Z]{13})$",
				"description": "GSTIN of e-Commerce operator"
			},
			"IgstOnIntra": {
				"type": "string",
				"minLength": 1,
				"maxLength": 1,
				"enum": [
					"Y",
					"N"
				],
				"pattern": "^([Y|N]{1})$",
				"description": "Y- indicates the supply is intra state but chargeable to IGST"
			}
		},
		"required": [
			"TaxSch",
			"SupTyp"
		]
	},
	"DocDtls": {
		"type": "object",
		"properties": {
			"Typ": {
				"type": "string",
				"minLength": 3,
				"maxLength": 3,
				"enum": [
					"INV",
					"CRN",
					"DBN"
				],
				"pattern": "(?i)^((INV)|(CRN)|(DBN))$",
				"description": "Document Type: INVOICE, CREDIT NOTE, DEBIT NOTE"
			},
			"No": {
				"type": "string",
				"minLength": 1,
				"maxLength": 16,
				"pattern": "^([a-zA-Z1-9]{1}[a-zA-Z0-9\/-]{0,15})$",
				"description": "Document Number"
			},
			"Dt": {
				"type": "string",
				"minLength": 10,
				"maxLength": 10,
				"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
				"description": "Document Date"
			}
		},
		"required": [
			"Typ",
			"No",
			"Dt"
		]
	},
	"SellerDtls": {
		"type": "object",
		"properties": {
			"Gstin": {
				"type": "string",
				"minLength": 15,
				"maxLength": 15,
				"pattern": "([0-9]{2}[0-9A-Z]{13})",
				"description": "GSTIN of supplier"
			},
			"LglNm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Legal Name"
			},
			"TrdNm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Tradename"
			},
			"Addr1": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Building/Flat no, Road/Street"
			},
			"Addr2": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address 2 of the supplier (Floor no., Name of the premises/building)"
			},
			"Loc": {
				"type": "string",
				"minLength": 3,
				"maxLength": 50,
				"pattern": "^([^\\\"])*$",
				"description": "Location"
			},
			"Pin": {
				"type": "number",
				"minimum": 100000,
				"maximum": 999999,
				"description": "Pincode"
			},
			"Stcd": {
				"type": "string",
				"minLength": 1,
				"maxLength": 2,
				"pattern": "^(?!0+$)([0-9]{1,2})$",
				"description": "State Code of the supplier. Refer the master"
			},
			"Ph": {
				"type": "String",
				"minLength": 6,
				"maxLength": 12,
				"pattern": "^([0-9]{6,12})$",
				"description": "Phone or Mobile No."
			},
			"Em": {
				"type": "string",
				"minLength": 6,
				"maxLength": 100,
				"pattern": "^[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+$",
				"description": "Email-Id"
			}
		},
		"required": [
			"Gstin",
			"LglNm",
			"Addr1",
			"Loc",
			"Pin",
			"Stcd"
		]
	},
	"BuyerDtls": {
		"type": "object",
		"properties": {
			"Gstin": {
				"type": "string",
				"minLength": 3,
				"maxLength": 15,
				"pattern": "^(([0-9]{2}[0-9A-Z]{13})|URP)$",
				"description": "GSTIN of buyer , URP if exporting"
			},
			"LglNm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Legal Name"
			},
			"TrdNm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Trade Name"
			},
			"Pos": {
				"type": "string",
				"minLength": 1,
				"maxLength": 2,
				"pattern": "^(?!0+$)([0-9]{1,2})$",
				"description": "State code of Place of supply. If POS lies outside the country, the code shall be 96."
			},
			"Addr1": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address 1 of the buyer. (Building/Flat no., Road/Street etc.)"
			},
			"Addr2": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address 2 of the buyer. (Floor no., Name of the premises/ building)"
			},
			"Loc": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Location"
			},
			"Pin": {
				"type": " number",
				"minimum": 100000,
				"maximum": 999999,
				"description": "Pincode"
			},
			"Stcd": {
				"type": "string",
				"minLength": 1,
				"maxLength": 2,
				"pattern": "^(?!0+$)([0-9]{1,2})$",
				"description": "State Code of the buyer. Refer the master"
			},
			"Ph": {
				"type": "String",
				"minLength": 6,
				"maxLength": 12,
				"pattern": "^([0-9]{6,12})$",
				"description": "Phone or Mobile No."
			},
			"Em": {
				"type": "string",
				"minLength": 6,
				"maxLength": 100,
				"pattern": "^[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+$",
				"description": "Email-Id"
			}
		},
		"required": [
			"Gstin",
			"LglNm",
			"Pos",
			"Addr1",
			"Loc",
			"Stcd"
		]
	},
	"DispDtls": {
		"type": "object",
		"properties": {
			"Nm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Name of the company from which the goods are dispatched"
			},
			"Addr1": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address 1 of the entity from which goods are dispatched. (Building/Flat no.Road/Street etc.)"
			},
			"Addr2": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address 2 of the entity from which goods are dispatched. (Floor no., Name of the premises/building)"
			},
			"Loc": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Location"
			},
			"Pin": {
				"type": "number",
				"minimum": 100000,
				"maximum": 999999,
				"description": "Pincode"
			},
			"Stcd": {
				"type": "string",
				"minLength": 1,
				"maxLength": 2,
				"pattern": "^(?!0+$)([0-9]{1,2})$",
				"description": "State Code. Refer the master"
			}
		},
		"required": [
			"Nm",
			"Addr1",
			"Loc",
			"Pin",
			"Stcd"
		]
	},
	"ShipDtls": {
		"type": "object",
		"properties": {
			"Gstin": {
				"type": "string",
				"maxLength": 15,
				"minLength": 3,
    						"pattern": "^(([0-9]{2}[0-9A-Z]{13})|URP)$",
				"description": "GSTIN of entity to whom goods are shipped"
			},
			"LglNm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Legal Name"
			},
			"TrdNm": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Trade Name"
			},
			"Addr1": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address1 of the entity to whom the supplies are shipped to. (Building/Flat no., Road/Street etc.)"
			},
			"Addr2": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Address 2 of the entity to whom the supplies are shipped to. (Floor no., Name of the premises/building)."
			},
			"Loc": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Place (City,Town,Village) entity to whom the supplies are shipped to."
			},
			"Pin": {
				"type": "number",
				"minimum": 100000,
				"maximum": 999999,
				"description": "Pincode"
			},
			"Stcd": {
				"type": "string",
				"minLength": 1,
				"maxLength": 2,
				"pattern": "^(?!0+$)([0-9]{1,2})$",
				"description": "State Code to which supplies are shipped to. Refer the master"
			}
		},
		"required": ["LglNm",
			"Addr1",
			"Loc",
			"Pin",
			"Stcd"
		]
	},
	"ItemList": [{
		"type": "object",
		"properties": {
			"SlNo": {
				"type": "string",
				"minLength": 1,
				"maxLength": 6,
				"pattern": "^([0-9]{1,6})$",
				"description": "Serial No. of Item"
			},
			"PrdDesc": {
				"type": "string",
				"minLength": 3,
				"maxLength": 300,
				"pattern": "^([^\\\"])*$",
				"description": "Product Description"
			},
			"IsServc": {
				"type": "string",
				"minLength": 1,
				"maxLength": 1,
				"enum": [
					"Y",
					"N"
				],
				"pattern": "^([Y|N]{1})$",
				"description": "Specify whether the supply is service or not. Specify Y-for Service"
			},
			"HsnCd": {
				"type": "string",
				"minLength": 4,
				"maxLength": 8,
				"pattern": "^(?!0+$)([0-9]{4}|[0-9]{6}|[0-9]{8})$",
				"description": "HSN Code. Refer Master"
			},
			"Barcde": {
				"type": "string",
				"minLength": 3,
				"maxLength": 30,
				"pattern": "^([^\\\"])*$",
				"description": "Bar Code"
			},
			"Qty": {
				"type": "number",
				"minimum": 0,
				"maximum": 9999999999.999,
				"description": "Quantity"
			},
			"FreeQty": {
				"type": "number",
				"minimum": 0,
				"maximum": 9999999999.999,
				"description": "Free Quantity"
			},
			"Unit": {
				"type": "string",
				"minLength": 3,
				"maxLength": 8,
				"pattern": "^([A-Z|a-z]{3,8})$",
				"description": "Unit. Refer the master"
			},
			"UnitPrice": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.999,
				"description": "Unit Price - Rate"
			},
			"TotAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Gross Amount (Unit Price * Quantity)"
			},
			"Discount": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Discount"
			},
			"PreTaxVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Pre tax value"
			},
			"AssAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Taxable Value (Total Amount -Discount)"
			},
			"GstRt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999.999,
				"description": "The GST rate, represented as percentage that applies to the invoiced item. It will IGST rate only."
			},
			"IgstAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": " Amount of IGST payable."
			},
			"CgstAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": " Amount of CGST payable."
			},
			"SgstAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": " Amount of SGST payable."
			},
			"CesRt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999.999,
				"description": "Cess Rate"
			},
			"CesAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Cess Amount(Advalorem) on basis of rate and quantity of item"
			},
			"CesNonAdvlAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Cess Non-Advol Amount"
			},
			"StateCesRt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999.999,
				"description": "State CESS Rate"
			},
			"StateCesAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "State CESS Amount"
			},
			"StateCesNonAdvlAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "State CESS Non Adval Amount"
			},
			"OthChrg": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Other Charges"
			},
			"TotItemVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Total Item Value = Assessable Amount + CGST Amt + SGST Amt + Cess Amt + CesNonAdvlAmt + StateCesAmt + StateCesNonAdvlAmt+Otherchrg"
			},
			"OrdLineRef": {
				"type": "string",
				"minLength": 1,
				"maxLength": 50,
				"pattern": "^([^\\\"])*$",
				"description": "Order line reference"
			},
			"OrgCntry": {
				"type": "string",
				"minLength": 2,
				"maxLength": 2,
				"pattern": "^([A-Z|a-z]{2})$",
				"description": "Origin Country. Refer Master"
			},
			"PrdSlNo": {
				"type": "string",
				"minLength": 1,
				"maxLength": 20,
				"pattern": "^([^\\\"])*$",
				"description": "Serial number in case of each item having a unique number."
			},
			"BchDtls": {
				"type": "object",
				"properties": {
					"Nm": {
						"type": "string",
						"minLength": 3,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Batch number"
					},
					"ExpDt": {
						"type": "string",
						"maxLength": 10,
						"minLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description": "Batch Expiry Date"
					},
					"WrDt": {
						"type": "string",
						"maxLength": 10,
						"minLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description": "Warranty Date"
					}
				},
				"required": [
					"Nm"
				]
			},
			"AttribDtls": [{
				"type": "object",
				"properties": {
					"Nm": {
						"type": "string",
						"minLength": 1,
						"maxLength": 100,
						"pattern": "^([^\\\"])*$",
						"description": "Attribute name of the item"
					},
					"Val": {
						"type": "string",
						"minLength": 1,
						"maxLength": 100,
						"pattern": "^([^\\\"])*$",
						"description": "Attribute value of the item"
					}
				}
			}]
		},
		"required": [
			"SlNo",
			"IsServc",
			"HsnCd",
			"UnitPrice",
			"TotAmt",
			"AssAmt",
			"GstRt",
			"TotItemVal"
		]
	}],
	"ValDtls": {
		"type": "object",
		"properties": {
			"AssVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Total Assessable value of all items"
			},
			"CgstVal": {
				"type": "number",
				"maximum": 99999999999999.99,
				"minimum": 0,
				"description": "Total CGST value of all items"
			},
			"SgstVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Total SGST value of all items"
			},
			"IgstVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Total IGST value of all items"
			},
			"CesVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Total CESS value of all items"
			},
			"StCesVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Total State CESS value of all items"
			},
			"Discount": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Discount"
			},
			"OthChrg": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Other Charges"
			},
			"RndOffAmt": {
				"type": "number",
				"minimum": -99.99,
				"maximum": 99.99,
				"description": "Rounded off amount"
			},
			"TotInvVal": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Final Invoice value "
			},
			"TotInvValFc": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Final Invoice value in Additional Currency"
			}
		},
		"required": [
			"AssVal",
			"TotInvVal"
		]
	},
	"PayDtls": {
		"type": "object",
		"properties": {
			"Nm": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Payee Name"
			},
			"AccDet": {
				"type": "string",
				"minLength": 1,
				"maxLength": 18,
				"pattern": "^([^\\\"])*$",
				"description": "Bank account number of payee"
			},
			"Mode": {
				"type": "string",
				"minLength": 1,
				"maxLength": 18,
				"pattern": "^([^\\\"])*$",
				"description": "Mode of Payment: Cash, Credit, Direct Transfer"
			},
			"FinInsBr": {
				"type": "string",
				"minLength": 1,
				"maxLength": 11,
				"pattern": "^([^\\\"])*$",
				"description": "Branch or IFSC code"
			},
			"PayTerm": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Terms of Payment"
			},
			"PayInstr": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Payment Instruction"
			},
			"CrTrn": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Credit Transfer"
			},
			"DirDr": {
				"type": "string",
				"minLength": 1,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Direct Debit"
			},
			"CrDay": {
				"type": "number",
				"minimum": 0,
				"maximum": 9999,
				"description": "Credit Days"
			},
			"PaidAmt": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "The sum of amount which have been paid in advance."
			},
			"PaymtDue": {
				"type": "number",
				"minimum": 0,
				"maximum": 99999999999999.99,
				"description": "Outstanding amount that is required to be paid."
			}
		}
	},
	"RefDtls": {
		"type": "object",
		"properties": {
			"InvRm": {
				"type": "string",
				"maxLength": 100,
				"minLength": 3,
				"pattern": "^([^\\\"])*$",
				"description": "Remarks/Note"
			},
			"DocPerdDtls": {
				"type": "object",
				"properties": {
					"InvStDt": {
						"type": "string",
						"maxLength": 10,
						"minLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description": "Invoice Period Start Date"
					},
					"InvEndDt": {
						"type": "string",
						"maxLength": 10,
						"minLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description": "Invoice Period End Date"
					}
				},
				"required": [
					" InvStDt ",
					" InvEndDt "
				]
			},
			"PrecDocDtls": [{
				"type": "object",
				"properties": {
					"InvNo": {
						"type": "string",
						"minLength": 1,
						"maxLength": 16,
						"pattern": "^[1-9a-zA-Z]{1}[0-9a-zA-Z\/-]{1,15}$",
						"description": "Reference of original invoice, if any."
					},
					"InvDt": {
						"type": "string",
						"maxLength": 10,
						"minLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description": "Date of preceding invoice"
					},
					"OthRefNo": {
						"type": "string",
						"minLength": 1,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Other Reference"
					}
				}
			}],
			"required": [
				"InvNo",
				"InvDt"
			],
			"ContrDtls": [{
				"type": "object",
				"properties": {
					"RecAdvRefr": {
						"type": "string",
						"minLength": 1,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Receipt Advice No."
					},
					"RecAdvDt": {
						"type": "string",
						"minLength": 10,
						"maxLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description": "Date of receipt advice"
					},
					"TendRefr": {
						"type": "string",
						"minLength": 1,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Lot/Batch Reference No."
					},
					"ContrRefr": {
						"type": "string",
						"minLength": 1,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Contract Reference Number"
					},
					"ExtRefr": {
						"type": "string",
						"minLength": 1,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Any other reference"
					},
					"ProjRefr": {
						"type": "string",
						"minLength": 1,
						"maxLength": 20,
						"pattern": "^([^\\\"])*$",
						"description": "Project Reference Number"
					},
					"PORefr": {
						"type": "string",
						"minLength": 1,
						"maxLength": 16,
						"pattern": "^([^\\\"])*$",
						"description": " PO Reference Number"
					},
					"PORefDt": {
						"type": "string",
						"minLength": 10,
						"maxLength": 10,
						"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
						"description ": "POReferencedate "
					}
				}
			}]
		}
	},
	"AddlDocDtls ": [{
		"type ": "object ",
		"properties ": {
			"Url ": {
				"type ": "string ",
				"minLength ": 3,
				"maxLength ": 100,
				"pattern ": "^([^\\\"])*$",
				"description ": "SupportingdocumentURL "
			},
			"Docs ": {
				"type ": "string ",
				"minLength ": 3,
				"maxLength ": 1000,
				"pattern ": "^([^\\\"])*$",
				"description": "Supporting document in Base64 Format"
			},
			"Info": {
				"type": "string",
				"minLength": 3,
				"maxLength": 1000,
				"pattern": "^([^\\\"])*$",
				"description": "Any additional information"
			}
		}
	}],
	"ExpDtls": {
		"type": "object",
		"properties": {
			"ShipBNo": {
				"type": "string",
				"minLength": 1,
				"maxLength": 20,
				"pattern": "^([^\\\"])*$",
				"description": "Shipping Bill No."
			},
			"ShipBDt": {
				"type": "string",
				"minLength": 10,
				"maxLength": 10,
				"pattern": "^[0-3][0-9]\/[0-1][0-9]\/[2][0][1-2][0-9]$",
				"description": "Shipping Bill Date"
			},
			"Port": {
				"type": "string",
				"minLength": 2,
				"maxLength": 10,
				"pattern": "^[0-9|A-Z|a-z]{2,10}$",
				"description": "Port Code. Refer the master"
			},
			"RefClm": {
				"type": "string",
				"minLength": 1,
				"maxLength": 1,
				"pattern": "^([Y|N]{1})$",
				"description": "Claiming Refund. Y/N"
			},
			"ForCur": {
				"type": "string",
				"minLength": 3,
				"maxLength": 16,
				"pattern": "^[A-Z|a-z]{3,16}$",
				"description": "Additional Currency Code. Refer the master"
			},
			"CntCode": {
				"type": "string",
				"minLength": 2,
				"maxLength": 2,
				"pattern": "^([A-Z]{2})$",
				"description": "Country Code. Refer the master"
			},
			"ExpDuty": {
				"type": "number",
				"minimum": 0,
				"maximum": 999999999999.99,
				"description": "Export Duty"
			}
		}
	},
	"EwbDtls": {
		"type": "object",
		"properties": {
			"TransId": {
				"type": "string",
				"minLength": 15,
				"maxLength": 15,
				"pattern": "^([0-9]{2}[0-9A-Z]{13})$",
				"description": "Transin/GSTIN"
			},
			"TransName": {
				"type": "string",
				"minLength": 3,
				"maxLength": 100,
				"pattern": "^([^\\\"])*$",
				"description": "Name of the transporter"
			},
			"TransMode": {
				"type": "string",
				"maxLength": 1,
				"minLength": 1,
				"enum": [
					"1",
					"2",
					"3",
					"4"
				],
				"pattern": "^([1-4]{1})?$",
				"description": "Mode of transport (Road-1, Rail-2, Air-3, Ship-4)"
			},
			"Distance": {
				"type": "number",
				"minimum": 0,
				"maximum": 4000,
				"description": " Distance between source and destination PIN codes"
			},
			"TransDocNo": {
				"type": "string",
				"minLength": 1,
				"maxLength": 15,
				"pattern": "^([a-zA-Z0-9\/-]{1,15})$",
				"description": "Tranport Document Number"
			},
			"TransDocDt": {
				"type": "string",
				"minLength": 10,
				"maxLength": 10,
				"pattern": "^[a-zA-Z0-9]{1}[a-zA-Z0-9-/]*$",
				"description": "Transport Document Date"
			},
			"VehNo": {
				"type": "string",
				"minLength": 4,
				"maxLength": 20,
				"pattern": "^([A-Z|a-z|0-9]{4,20})$",
				"description": "Vehicle Number"
			},
			"VehType": {
				"type": "string",
				"minLength": 1,
				"maxLength": 1,
				"enum": [
					"O",
					"R"
				],
				"pattern": "^([O|R]{1})$",
				"description": "Whether O-ODC or R-Regular "
			}
		},
		"required": [
			"Distance"
		]
	},
	"required": [
		"Version",
		"TranDtls",
		"DocDtls",
		"SellerDtls",
		"BuyerDtls",
		"ItemList",
		"ValDtls"
	]
}

json_string = """
{
  "Version": "1.1",
  "TranDtls": {
    "TaxSch": "GST",
    "SupTyp": "B2B",
    "RegRev": "Y",
    "EcmGstin": null,
    "IgstOnIntra": "N"
  },
  "DocDtls": {
    "Typ": "INV",
    "No": "DOC/001",
    "Dt": "18/08/2020"
  },
  "SellerDtls": {
    "Gstin": "37ARZPT4384Q1MT",
    "LglNm": "NIC company pvt ltd",
    "TrdNm": "NIC Industries",
    "Addr1": "5th block, kuvempu layout",
    "Addr2": "kuvempu layout",
    "Loc": "GANDHINAGAR",
    "Pin": 518001,
    "Stcd": "37",
    "Ph": "9000000000",
    "Em": "abc@gmail.com"
  },
  "BuyerDtls": {
    "Gstin": "29AWGPV7107B1Z1",
    "LglNm": "XYZ company pvt ltd",
    "TrdNm": "XYZ Industries",
    "Pos": "12",
    "Addr1": "7th block, kuvempu layout",
    "Addr2": "kuvempu layout",
    "Loc": "GANDHINAGAR",
    "Pin": 562160,
    "Stcd": "29",
    "Ph": "91111111111",
    "Em": "xyz@yahoo.com"
  },
  "DispDtls": {
    "Nm": "ABC company pvt ltd",
    "Addr1": "7th block, kuvempu layout",
    "Addr2": "kuvempu layout",
    "Loc": "Banagalore",
    "Pin": 562160,
    "Stcd": "29"
  },
  "ShipDtls": {
    "Gstin": "29AWGPV7107B1Z1",
    "LglNm": "CBE company pvt ltd",
    "TrdNm": "kuvempu layout",
    "Addr1": "7th block, kuvempu layout",
    "Addr2": "kuvempu layout",
    "Loc": "Banagalore",
    "Pin": 451610,
    "Stcd": "29"
  },
  "ItemList": [
    {
      "SlNo": "1",
      "PrdDesc": "Rice",
      "IsServc": "N",
      "HsnCd": "1001",
      "Barcde": "123456",
      "Qty": 100.345,
      "FreeQty": 10,
      "Unit": "BAG",
      "UnitPrice": 99.545,
      "TotAmt": 9988.84,
      "Discount": 10,
      "PreTaxVal": 1,
      "AssAmt": 9978.84,
      "GstRt": 12.0,
      "IgstAmt": 1197.46,
      "CgstAmt": 0,
      "SgstAmt": 0,
      "CesRt": 5,
      "CesAmt": 498.94,
      "CesNonAdvlAmt": 10,
      "StateCesRt": 12,
      "StateCesAmt": 1197.46,
      "StateCesNonAdvlAmt": 5,
      "OthChrg": 10,
      "TotItemVal": 12897.7,
      "OrdLineRef": "3256",
      "OrgCntry": "AG",
      "PrdSlNo": "12345",
      "BchDtls": {
        "Nm": "123456",
        "ExpDt": "01/08/2020",
        "WrDt": "01/09/2020"
      },
      "AttribDtls": [
        {
          "Nm": "Rice",
          "Val": "10000"
        }
      ]
    }
  ],
  "ValDtls": {
    "AssVal": 9978.84,
    "CgstVal": 0,
    "SgstVal": 0,
    "IgstVal": 1197.46,
    "CesVal": 508.94,
    "StCesVal": 1202.46,
    "Discount": 10,
    "OthChrg": 20,
    "RndOffAmt": 0.3,
    "TotInvVal": 12908,
    "TotInvValFc": 12897.7
  },
  "PayDtls": {
    "Nm": "ABCDE",
    "AccDet": "5697389713210",
    "Mode": "Cash",
    "FinInsBr": "SBIN11000",
    "PayTerm": "100",
    "PayInstr": "Gift",
    "CrTrn": "test",
    "DirDr": "test",
    "CrDay": 100,
    "PaidAmt": 10000,
    "PaymtDue": 5000
  },
  "RefDtls": {
    "InvRm": "TEST",
    "DocPerdDtls": {
      "InvStDt": "01/08/2020",
      "InvEndDt": "01/09/2020"
    },
    "PrecDocDtls": [
      {
        "InvNo": "DOC/002",
        "InvDt": "01/08/2020",
        "OthRefNo": "123456"
      }
    ],
    "ContrDtls": [
      {
        "RecAdvRefr": "Doc/003",
        "RecAdvDt": "01/08/2020",
        "TendRefr": "Abc001",
        "ContrRefr": "Co123",
        "ExtRefr": "Yo456",
        "ProjRefr": "Doc-456",
        "PORefr": "Doc-789",
        "PORefDt": "01/08/2020"
      }
    ]
  },
  "AddlDocDtls": [
    {
      "Url": "https://einv-apisandbox.nic.in",
      "Docs": "Test Doc",
      "Info": "Document Test"
    }
  ],
  "ExpDtls": {
    "ShipBNo": "A-248",
    "ShipBDt": "01/08/2020",
    "Port": "INABG1",
    "RefClm": "N",
    "ForCur": "AED",
    "CntCode": "AE",
    "ExpDuty": null
  },
  "EwbDtls": {
    "TransId": "12AWGPV7107B1Z1",
    "TransName": "XYZ EXPORTS",
    "Distance": 100,
    "TransDocNo": "DOC01",
    "TransDocDt": "18/08/2020",
    "VehNo": "ka123456",
    "VehType": "R",
    "TransMode": "1"
  }
}
"""

if is_valid_json(json_string, json_schema):
    print("JSON is valid")
else:
    print("JSON is not valid")

JSON decoding failed: Expecting value at line 46
Error occurred at line 46:
    "Pin": jaipur,
JSON is not valid
